In [ ]:
# This sets up the Jupyter version of Dash to allow the dashboard to run inside Jupyter Notebooks
from jupyter_dash import JupyterDash

# Import necessary components from Dash and related libraries
import dash_leaflet as dl  # For interactive maps
from dash import dcc, html, dash_table  # Core Dash components: dropdowns, html tags, and tables
from dash.dependencies import Input, Output  # For interactivity callbacks
import base64  # To encode images for embedding
import pandas as pd  # Data handling library
import plotly.express as px  # For creating plots and charts

# Import the custom CRUD class to handle MongoDB operations
from animal_shelter import AnimalShelter

# Database credentials 
username = "aacuser"
password = "SNHU1234"

# Connect to MongoDB using the CRUD module
db = AnimalShelter(username, password)

# Read all records from the database
df = pd.DataFrame.from_records(db.read({}))
# Drop the MongoDB _id field to avoid datatype errors when displaying data
df.drop(columns=['_id'], inplace=True)

# Load and encode an image file (logo) to embed in the dashboard header
image_filename = 'my-image.png'  # Ensure this file exists in your project folder
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Initialize the Dash app for use inside Jupyter Notebook
app = JupyterDash(__name__)

# Define the app layout, which controls the structure and appearance of the dashboard
app.layout = html.Div([
    # Page title, centered and bold
    html.Center(html.B(html.H1('Lauren-Ann Javier: CS-340 Dashboard'))),

    # Embed the logo image below the title with 50% width scaling
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '50%'})),

    html.Hr(),  # Horizontal line for separation

    # Rescue type filter: radio buttons to allow user to filter data by rescue category
    html.Div([
        html.Label("Rescue Type Filter:"),  # Label for accessibility
        dcc.RadioItems(
            id='filter-type',  # Component ID for callback targeting
            options=[
                {'label': 'All', 'value': 'All'},
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain/Wilderness'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster/Tracking'}
            ],
            value='All',  # Default selected option
            inline=True  # Display radio items horizontally
        )
    ], style={'padding': '10px'}),  # Add padding around the filter for spacing

    html.Hr(),

    # DataTable showing the filtered animal shelter data
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],  # Columns from dataframe
        data=df.to_dict('records'),  # Data passed as list of dicts
        page_size=10,  # Number of rows per page
        row_selectable='single',  # Allow selection of a single row at a time
        sort_action="native",  # Allow native column sorting
        style_table={'overflowX': 'auto'},  # Allow horizontal scroll if needed
        style_data_conditional=[{  # Conditional styling to highlight the 'breed' column
            'if': {'column_id': 'breed'},
            'background_color': '#D2F3FF'
        }]
    ),

    html.Br(),
    html.Hr(),

    # Container for pie chart and map, arranged side-by-side with flexbox layout
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),  # Placeholder for pie chart
        html.Div(id='map-id', className='col s12 m6')  # Placeholder for map
    ])
])

# Callback to update the data table based on selected rescue type filter
@app.callback(
    Output('datatable-id', 'data'),  # Output updates the table's data prop
    Input('filter-type', 'value')  # Input listens to filter radio buttons
)
def update_table(filter_type):
    # Filter dataframe according to selected rescue type
    if filter_type == 'All':
        filtered_df = df  # Show all data if 'All' selected
    else:
        filtered_df = df[df['rescueType'] == filter_type]  # Filter by rescueType column
    return filtered_df.to_dict('records')  # Return filtered data to update table

# Callback to update pie chart when the table data changes (filtered)
@app.callback(
    Output('graph-id', 'children'),  # Output is the content inside graph-id div
    Input('datatable-id', 'data')  # Input is the current data in the data table
)
def update_graph(data):
    dff = pd.DataFrame(data)  # Convert table data back to dataframe
    if dff.empty:
        return html.P("No data available for the selected filter.")  # Display message if empty
    # Create pie chart of animal breeds in filtered data
    fig = px.pie(dff, names='breed', title='Preferred Animals by Breed')
    return dcc.Graph(figure=fig)  # Render the pie chart

# Callback to update the Leaflet map based on the selected row in the data table
@app.callback(
    Output('map-id', 'children'),  # Output to map container
    [Input('datatable-id', 'data'),  # Input data to know current rows visible
     Input('datatable-id', 'selected_rows')]  # Input which row is selected
)
def update_map(rows, selected_rows):
    # If no data or no selection, prompt user to select a row
    if not rows or selected_rows is None or len(selected_rows) == 0:
        return html.P("Select a row in the table to see the location on the map.")

    dff = pd.DataFrame(rows)  # Convert data back to dataframe
    row = selected_rows[0]  # Get the index of selected row

    # Extract latitude and longitude from columns 13 and 14 respectively
    lat = dff.iloc[row, 13]
    lon = dff.iloc[row, 14]
    breed = dff.iloc[row, 4]
    name = dff.iloc[row, 9]

    # Return a Leaflet map centered on the selected animal's location with a marker
    return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),  # Base map layer
        dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(breed),  # Tooltip shows breed when hovered
            dl.Popup([
                html.H1("Animal Name"),  # Popup title
                html.P(name)  # Popup content showing animal name
            ])
        ])
    ])

# Run the server in debug mode for live updates during development
app.run_server(debug=True)
